# Spam classification with NLTK and Logistic Regression


Here I have build an spam classifier they are so many ways but I have made this one and since I am a beginner so suggestions are most welcome

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

* If we choose supervised learning then we should take X = features of mail and Y= spam or not spam
* feature of X will be list of words included for spam or not spam.
* firstly I have taken NLTK implementation ,which have so many features like tokenizers,stemmers,stopward
* numpy is used for multiplication of matix
* pandas for reading the csv and a little of pre-processing  and sklearn for train-test splitting, confusion matrix and losgistic regression.





In [ ]:
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopword
import unicodedata
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
_MOST_COMMON_COUNT = 1200

In [ ]:
data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv',encoding='Windows-1252')


Before we start with coding stuff, let us first analyse the data set.In our first column we have found our data as spam and ham(not spam). It can be easily seen from the below table that text is in the form of string and we need to first vectorize the string in the numerical list of list, so basically this is used for learning vector representations of words.

In [ ]:
data = data.loc[:,['v1','v2']]
data = data.dropna()
data.head()

**VECTORIZATION**

we define a fixed length vector where each entry corresponds to a word in our pre-defined dictionary of words. The size of the vector equals the size of the dictionary. Then, for representing a text using this vector, we count how many times each word of our dictionary appears in the text and we put this number in the corresponding vector entry.

For example, if our dictionary contains the words {ABDULKALAM, is, the, not, great}, and we want to vectorize the text “ABDULKALAM is great”, we would have the following vector: (1, 1, 0, 0, 1)

To improve this representation, you can use some more advanced techniques like removing stopwords, lemmatizing words, using n-grams or using tf-idf instead of counts.

**STOPWARDS**

 stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine wants to ignore.We do not want these words to take space in our database, or taking up valuable processing time. For this, we can remove them easily, by storing a list of words that you consider to stop words. NLTK(Natural Language Toolkit) in python has a list of stopwords stored in 16 different languages. You can find them in the nltk_data directory.

In [ ]:
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")
def normalize_text(text):
    tokens = wordpunct_tokenize(text)
    tokens = [
        unicodedata.normalize('NFD', stemmer.stem(t) )
        for t in tokens 
        if t not in stop_words
    ]
    sentence = ' '.join(tokens)
    return sentence

In [ ]:
data.loc[:, 'v2'] = data.loc[:, 'v2'].apply(normalize_text)


**In order to check which words are more frequent**

Now all messages are normalized, reducing the total amount of distinct words that we have in our messages and now we are ready to identify which words are more frequent

In [ ]:
counter = Counter()
for line in data.loc[:, 'v2']:
    counter.update(wordpunct_tokenize(line))

In [ ]:
most_common = [w[0] for w in counter.most_common(_MOST_COMMON_COUNT)]
word_map = {str(word): index for (word, index) in zip(most_common, range(len(most_common)))}

In [ ]:
def transform_to_freq(text):
    features = np.zeros(_MOST_COMMON_COUNT)
    for word in wordpunct_tokenize(text):
        if str(word) in word_map:
            word_index = word_map[str(word)]
            features[word_index] = features[word_index] + 1
    return features.tolist()

In [ ]:
X = np.array([transform_to_freq(line) for line in data.loc[:, 'v2']])
y = (data.loc[:, 'v1'].values == 'spam').reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
regressor = LogisticRegression(solver='lbfgs').fit(X_train, y_train.ravel())
predictions = regressor.predict(X_test)

![https://www.google.com/url?sa=i&url=https%3A%2F%2Ftowardsdatascience.com%2Funderstanding-confusion-matrix-a9ad42dcfd62&psig=AOvVaw2wmH8em8FLKpB8zAcUp1k1&ust=1598809478631000&source=images&cd=vfe&ved=0CAIQjRxqFwoTCPimi8v7wOsCFQAAAAAdAAAAABAD](http://)

you can go though this to get a desired picture of confusion matrix
Well I'tell you a short about this wht is **CONFUSION MATRIX**

A confusion matrix is a table used to describe the performance of a classifier or a data set for that truw values should be known



In [ ]:
cm = confusion_matrix(y_test, predictions)
cm

# Precision
of all the data set where we predicted what fraction y = 1 have actually spam mails

# Recall
of all the data set having actually spam mails what eaction did we correctly detect have spam mails

In [ ]:
 tn = cm[0][0] # True Negatives
fn = cm[1][0] # False Negatives
tp = cm[1][1] # True Positives
fp = cm[0][1] # False Positives

recall = tp / (tp + fn)
precision = tp / (tp + fp)
accuracy = (tp + tn) / (tn + fn + tp + fp)
f1_score = 2 * ((precision * recall) / (precision + recall))*100
f1_score